In [26]:
#Bibliotecas utilizadas
import pandas as pd
import numpy as np
import nltk
from nltk import FreqDist
from nltk.tokenize import sent_tokenize
import re
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report 
from sklearn import svm

#Definição de funções úteis
def RemoviStopWords(instancia):
    instancia = instancia.lower()
    stopwords = set(nltk.corpus.stopwords.words('english'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))
def Pre_Processing_data(frase):
    temp = frase
    temp = re.sub(r'[-./?!,":;()\'+*#&<>$~]',' ',temp)
    temp = re.sub('[-|0-9]',' ', temp)
    temp = RemoviStopWords(temp)
    return (temp)

In [4]:
msgs = pd.read_csv("Reviews.csv",encoding = "ISO-8859-1")
msgs.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
#Filtragem dos dados relevantes para elaboração da resolução do problema
#Apenas scores e comentário do usuário
scores = pd.DataFrame(msgs['Score'])
scores['Full_Text'] = msgs['Text']
scores.head()

,Score,Full_Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...


In [6]:
#Filtragem dos dados com scores ruins pra classificar as mensagens
worst_msgs = scores[scores['Score'] <= 2]
worst_msgs = worst_msgs.reset_index(drop=True)
worst_msgs.head()

,Score,Full_Text
0,1,Product arrived labeled as Jumbo Salted Peanut...
1,2,If you are looking for the secret ingredient i...
2,1,My cats have been happily eating Felidae Plati...
3,2,I love eating them and they are good for watch...
4,1,"The candy is just red , No flavor . Just plan..."


In [8]:
#Extraindo dados com scores ruins que contenham a palavra peanuts
Full_worst_Msgs = list(worst_msgs["Full_Text"])#Selecionando apenas o texto completo das mensagens
peanuts_bad=[] #Lista que armazena as mensagens com scores ruins que contenham a a palavra Peanut
no_peanuts_bad =[] #Lista que armazena as mensagens com scores ruins que não contenham a palavra Peanut

#Lista com classificação de cada msg (Possui a palavra peanut?) a ser utilizada pelo algoritmo futuramente
#Classes a serem usadas: Peanuts_Bad, Peanuts_Best e No_classified
peanuts_review_bad = []
no_peanuts_review_bad = []

#lista para fins comparativos ao final da proposta
n_col_has_peanuts_bad = [] 
bad_msgs_peanuts = []
msgs_no_peanuts = []
n_col = []

#Tratamento das mensagens, removendo Stop Words e retirando caracteres especiais
#Coleta de dados para uso no Algoritmo de Classificação
for ind in range(len(Full_worst_Msgs)):
    if ind == 30000: #Base de dados extensa, foi limitado por limitação do hardware utilizado para resolução do problema
        break
    temp = Full_worst_Msgs[ind] 
    if  temp.count('peanuts') >= 1 or temp.count('Peanuts') >=1 or temp.count('peanut') >= 1 or temp.count('Peanut') >= 1:
        bad_msgs_peanuts.append(temp)            #Para uso posterior, mensagem guardada antes de ser pré processada
        temp = Pre_Processing_data(temp)         #Pre processamento da mensagem
        peanuts_bad.append(temp)                 #Incluir mensagem na lista após pré processamento
        peanuts_review_bad.append("Peanuts_Bad") #Classificação
        n_col_has_peanuts_bad.append("Yes")      #Para uso posterior (Mensagem contém a palavra Peanuts?)
    else:
        msgs_no_peanuts.append(temp)                  #Para uso posterior
        temp = Pre_Processing_data(temp)              #Pre processamento da mensagem
        no_peanuts_bad.append(temp)                   #Incluir mensagem na lista após pré processamento
        no_peanuts_review_bad.append("No_classified") #Classificação
        n_col.append("No")                            #Para uso posterior (Mensagem contém a palavra Peanuts?)

Finish


In [9]:
#Extraindo dados com scores bons para classificar as mensagens
best_msgs = scores[scores['Score'] >= 3]
best_msgs = best_msgs.reset_index(drop=True)
best_msgs.head()

,Score,Full_Text
0,5,I have bought several of the Vitality canned d...
1,4,This is a confection that has been around a fe...
2,5,Great taffy at a great price. There was a wid...
3,4,I got a wild hair for taffy and ordered this f...
4,5,This saltwater taffy had great flavors and was...


In [14]:
#Extraindo dados com scores ruins que contenham a palavra peanuts
Full_best_Msgs = list(best_msgs["Full_Text"])#Selecionando apenas o texto completo das mensagens
peanuts_best=[]                              #Lista usada para armazenar as mensagens com scores bons e que contenham a palavra peanuts
#Lista com msgs apenas com resposta se a review com peanut e ruim ou nao
peanuts_review_best = []                     #Lista usada para armazenar se review é bom ou não
no_peanuts_review_best = []

best_msgs_peanuts = []                       #Lista usada para armazenar as mensagens antes de pré processá-las
n_col_has_peanuts_best = []                 #Lista usada para armazenar se as mensagens com scores bons contém a palavra peanuts

#Tratamento das mensagens, removendo Stop Words e retirando caracteres especiais
#Separando as mensagens com Scores bons que contém a palavra Peanuts
for ind in range(len(Full_best_Msgs)):
    if ind == 30000:
        break
    temp = Full_best_Msgs[ind]
    if temp.count('peanuts') >= 1 or temp.count('Peanuts') >=1 or temp.count('peanut') >= 1 or temp.count('Peanut') >= 1:
        best_msgs_peanuts.append(temp)
        temp = Pre_Processing_data(temp)
        peanuts_best.append(temp)
        peanuts_review_best.append("Peanuts_good") #Classificação
        n_col_has_peanuts_best.append("Yes")       #Para uso posterior (Mensagem contém a palavra Peanuts?)

#Os passos a seguir são os mesmos realizados para mensagens com scores ruins feitas anteriormente
    else:
        msgs_no_peanuts.append(temp)
        temp = Pre_Processing_data(temp)
        no_peanuts_bad.append(temp)
        no_peanuts_review_bad.append("No_classified") 
        n_col.append("No")

Finish


In [15]:
#Unindo lista com mensagens e suas devidas classificações dadas pelos Database
#Mensangens com scores bons e que contém a palavra peanuts Classificado como: Peanuts_Good
peanuts_best_set  = list(zip(peanuts_best,peanuts_review_best))

#Mensangens com scores ruins e que contém a palavra peanuts Classificado como: Peanuts_bad
peanuts_bad_set = list(zip(peanuts_bad,peanuts_review_bad))

#Mensangens que não contém a palavra peanuts não dependem dos scores, por isso são classificadas como: No_classified
no_peanuts = list(zip(no_peanuts_bad,no_peanuts_review_bad))

In [17]:
print(len(peanuts_best_set))
print(len(peanuts_bad_set))
print(len(no_peanuts))


684
586
88184


In [19]:
#Definindo conjunto de treinamento e conkunto de teste para uso do algoritmo SVM
training_corpus = peanuts_bad_set[0:int((len(peanuts_bad_set)*0.8))]+peanuts_best_set[0: int((len(peanuts_best_set)*0.8))] + no_peanuts[0:1900]
test_corpus = peanuts_bad_set[int((len(peanuts_bad_set)*0.8)):]+peanuts_best_set[int((len(peanuts_best_set)*0.8)):]+no_peanuts[1600:1700]
#Misturando itens da lista
random.shuffle(training_corpus)
random.shuffle(test_corpus)

In [20]:
# Preparando dados para uso do SVM model (Máquinas de Vetores de Suporte)
train_data = []
train_labels = []
for row in training_corpus:
    train_data.append(row[0])
    train_labels.append(row[1])

test_data = [] 
test_labels = [] 
for row in test_corpus:
    test_data.append(row[0]) 
    test_labels.append(row[1])

# Create vetores de características 
vectorizer = TfidfVectorizer(min_df=4, max_df=0.9)
# Treinamento
train_vectors = vectorizer.fit_transform(train_data)
# Aplicando modelo aos dados de teste
test_vectors = vectorizer.transform(test_data)

# Realizando a classificação com o SVM 
model = svm.SVC(kernel='linear') 
model.fit(train_vectors, train_labels) 
prediction = model.predict(test_vectors)
print (classification_report(test_labels, prediction))


               precision    recall  f1-score   support

No_classified       0.75      1.00      0.86       100
  Peanuts_Bad       0.77      0.69      0.73       118
 Peanuts_good       0.81      0.69      0.74       137

  avg / total       0.78      0.78      0.77       355



In [21]:
#print(len(test_msgs),len(n_col_has_peanuts))
test_msgs = best_msgs_peanuts[300:320] + bad_msgs_peanuts[300:320] + msgs_no_peanuts[300:320]
n_col_msgs = n_col_has_peanuts_best[300:320] + n_col_has_peanuts_bad[300:320] + n_col[300:320] 
test_msgs = pd.DataFrame(test_msgs)
test_msgs.rename(columns={0:'Full_Text'}, inplace=True)
#print(len(test_msgs),len(n_col_msgs))
test_msgs["have_peanut?"] = n_col_msgs
Clac = peanuts_review_best[300:320] + peanuts_review_bad[300:320] + no_peanuts_review_bad[300:320]
test_msgs["Classify by Amazon Customers"] = Clac
test_msgs

,Full_Text,have_peanut?,Classify by Amazon Customers
0,I kind of expected something that would taste ...,Yes,Peanuts_good
1,Tastes exactly like regular fatty creamy peanu...,Yes,Peanuts_good
2,This is really good. I am on weight watchers a...,Yes,Peanuts_good
3,I was impressed. It smells like peanut butter ...,Yes,Peanuts_good
4,"I found the taste to be satisfactory, especial...",Yes,Peanuts_good
5,It really does taste good! I don't like to me...,Yes,Peanuts_good
6,Wasn't sure how easy this product would be to ...,Yes,Peanuts_good
7,Used this product for baking. Finally could ma...,Yes,Peanuts_good
8,"""YOU MUST TRY THIS!"" Peanut butter lover, PB2 ...",Yes,Peanuts_good
9,I was a little hesitant to try this...how coul...,Yes,Peanuts_good


In [25]:
teste_data = best_msgs_peanuts[300:320] + bad_msgs_peanuts[300:320] + msgs_no_peanuts[300:320]
test_vectors = vectorizer.transform(teste_data) 
prediction = model.predict(test_vectors)
#print(len(Cla))
test_msgs["Classified_by_Algorithm"] = prediction
test_msgs

,Full_Text,have_peanut?,Classify by Amazon Customers,Classified_by_Algorithm
0,I kind of expected something that would taste ...,Yes,Peanuts_good,Peanuts_good
1,Tastes exactly like regular fatty creamy peanu...,Yes,Peanuts_good,Peanuts_good
2,This is really good. I am on weight watchers a...,Yes,Peanuts_good,Peanuts_good
3,I was impressed. It smells like peanut butter ...,Yes,Peanuts_good,Peanuts_good
4,"I found the taste to be satisfactory, especial...",Yes,Peanuts_good,Peanuts_good
5,It really does taste good! I don't like to me...,Yes,Peanuts_good,Peanuts_good
6,Wasn't sure how easy this product would be to ...,Yes,Peanuts_good,Peanuts_good
7,Used this product for baking. Finally could ma...,Yes,Peanuts_good,Peanuts_good
8,"""YOU MUST TRY THIS!"" Peanut butter lover, PB2 ...",Yes,Peanuts_good,Peanuts_good
9,I was a little hesitant to try this...how coul...,Yes,Peanuts_good,Peanuts_good
